# Here we extract table of parameters based on desired parameters

In [1]:
"""import packages you need"""
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler

from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

from IPython.display import clear_output


In [65]:
"""
This function filters the parameter combinations based on the angle of the engraved channel walls.  
Current conditions filter the data based on 
the target depth, angle, number of passes and accepted angle difference from the target angle.  
"""

def ParamTable_target(Data, angle, depth, ang_diff, no_pass):
        result = []
        indexes = []
        condition0 = (Data.Passes <= no_pass)
        condition1 = (Data.Bot_width => 0)
        condition2 = (Data.Top_width > 0) 
        condition3 = (Data.Depth < depth) 
        radian = np.arctan((Data.Top_width-Data.Bot_width)/(2* Data.Depth))
        condition4 = (np.abs(np.degrees(radian) - angle) <= ang_diff)
        index = (np.where(condition0 & condition1 & condition2 & condition3 &
                          condition4 ))[0]
        
        indexes.extend(index.tolist())

        for i in indexes:
            radian2 = np.arctan((Data.iloc[i].Top_width-Data.iloc[i].Bot_width)/(2* Data.iloc[i].Depth))
            case = np.r_[round(np.degrees(radian2),2), (Data.iloc[i].Frequency / 40. ), ((Data.iloc[i]).round(3))]
            result.append(case)  
        DATA = np.array(result)    
        df_results = pd.DataFrame(DATA,
                                  columns = 
    ['angle', 'Speed', 'Frequency','Amplitude', 'Passes', 'Focal_Position', 'Depth',
       'Top_width', 'Bot_width'])
        return df_results

In [194]:
### read the data base you want to filter some cases from
df = pd.read_csv('RawData/two_widths/resclGRID.csv')

result = ParamTable_target(df,30, 150, 2, 10)

### to include cases with positive values
result[(result > 0).all(1)]

result.to_csv(r'RawData/two_widths/TestData/Trap_Ang30_Grid.csv', header=True, index = False)

result

,angle,Speed,Frequency,Amplitude,Passes,Focal_Position,Depth,Top_width,Bot_width
0,31.80,10.00,400.0,0.5,10.0,91.676,139.209,602.162,429.564
1,31.06,11.25,450.0,0.5,10.0,91.676,133.755,574.510,413.420
2,31.76,12.50,500.0,0.5,10.0,91.676,122.590,556.876,405.090
3,28.12,13.75,550.0,0.4,10.0,91.676,149.791,442.184,282.062
4,29.97,15.00,600.0,0.4,10.0,91.676,131.368,432.834,281.342
...,...,...,...,...,...,...,...,...,...
136,31.20,40.00,1600.0,1.0,10.0,93.200,101.864,976.147,852.771
137,30.71,40.00,1600.0,1.1,10.0,92.819,107.668,1064.503,936.582
138,30.30,40.00,1600.0,1.1,10.0,93.200,102.373,1056.766,937.140
139,29.92,40.00,1600.0,1.2,10.0,92.819,109.212,1145.208,1019.528


In [69]:
### filtering cases from genrated data (GAN)
GAN_result = pd.read_csv('RawData/two_widths/GAN_tuned.csv')

result = ParamTable_target(GAN_result,8, 1000, 5, 90)

### to include cases with positive values
result[(result > 0).all(1)]

result.to_csv(r'RawData/two_widths/TestData/Trap_minAng_GAN.csv', header=True, index = False)
result

,angle,Speed,Frequency,Amplitude,Passes,Focal_Position,Depth,Top_width,Bot_width
0,4.87,39.939325,1597.573,0.389,15.143,93.166,134.482,395.243,372.347
1,4.87,39.939325,1597.573,0.389,15.143,93.166,134.482,395.243,372.347
2,8.54,27.687600,1107.504,0.282,21.383,91.699,42.447,439.729,426.977
3,4.87,39.939325,1597.573,0.389,15.143,93.166,134.482,395.243,372.347
4,8.54,27.687600,1107.504,0.282,21.383,91.699,42.447,439.729,426.977
5,8.54,27.687600,1107.504,0.282,21.383,91.699,42.447,439.729,426.977
6,11.51,28.777250,1151.090,0.715,14.875,92.312,103.042,597.853,555.874
7,8.54,27.687600,1107.504,0.282,21.383,91.699,42.447,439.729,426.977
8,12.27,39.549350,1581.974,0.608,15.166,92.493,91.117,499.440,459.803
9,11.51,28.777250,1151.090,0.715,14.875,92.312,103.042,597.853,555.874


In [200]:
"""When we need to filter data for channels with triangular cross-section"""
def ParamTable_triang(Data, angle, depth, ang_diff, no_pass):
        result = []
        indexes = []
        condition0 = (Data.Passes < no_pass)
        condition1 = (np.abs(Data.Bot_width) <= 10)
        condition2 = (Data.Top_width > 0) 
        condition3 = (Data.Depth > depth) 
        radian = np.arctan((Data.Top_width)/(2* Data.Depth))
        condition4 = (np.abs(np.degrees(radian) - angle) <= ang_diff)
        index = (np.where(condition0 & condition1 & condition2 & condition3 &
                          condition4 ))[0]
        indexes.extend(index.tolist())

        for i in indexes:
            radian2 = np.arctan((Data.iloc[i].Top_width-Data.iloc[i].Bot_width)/(2* Data.iloc[i].Depth))
            case = np.r_[round(np.degrees(radian2),2),Data.iloc[i].Frequency / 40., ((Data.iloc[i]).round(3))]
            result.append(case)  
        DATA = np.array(result)    
        df_results = pd.DataFrame(DATA, columns = 
    ['angle', 'Speed','Frequency', 'Amplitude', 'Passes', 'Focal_Position', 'Depth',
       'Top_width', 'Bot_width'])
        return df_results

In [203]:
Data = pd.read_csv('RawData/two_widths/resclGRID.csv')
df = Data.drop(['Speed'], axis = 'columns')


result = ParamTable_triang(df,20, 1000, 2, 120)

result[(result > 0).all(1)]

result.to_csv(r'RawData/two_widths/TestData/DeepTriang_Ang20Grid.csv', header=True, index = False)

result

,angle,Speed,Frequency,Amplitude,Passes,Focal_Position,Depth,Top_width,Bot_width
0,22.16,5.00,200.0,0.8,55.0,92.057,1059.863,854.296,-9.131
1,21.72,5.00,200.0,0.8,90.0,92.057,1101.395,869.525,-7.820
2,21.64,5.00,200.0,0.8,95.0,92.057,1103.284,871.873,-3.390
3,21.55,5.00,200.0,0.8,100.0,92.057,1105.356,874.159,0.954
4,21.47,5.00,200.0,0.8,105.0,92.057,1107.283,876.589,5.754
5,21.58,5.00,200.0,0.9,75.0,92.438,1157.358,920.674,5.229
6,21.55,5.00,200.0,0.9,80.0,92.438,1166.029,923.116,2.055
7,21.53,5.00,200.0,0.9,85.0,92.438,1173.899,925.343,-0.763
8,21.50,5.00,200.0,0.9,90.0,92.438,1180.950,927.115,-3.287
9,21.48,5.00,200.0,0.9,95.0,92.438,1187.143,928.839,-5.356


In [41]:
"""A function to report data from a dataframe for specific indexes"""
def Extract_index(Data, indexes):
        result = []
        for i in indexes:
            case = np.r_[i, ((Data.iloc[i]).round(3))]
            result.append(case)  
        DATA = np.array(result)    
        df_results = pd.DataFrame(DATA, columns = 
    ['index','Frequency', 'Speed', 'Amplitude', 'Passes', 'Focal_Position', 'Depth',
       'Angle'])
        return df_results

### Finding target dimensions for test

In [32]:
def ParamTable_TargetTrap(Data, depth, top_w, bot_w):
        result = []
        indexes = []

        condition0 = (Data.Bot_width > 0)
        condition1 = (np.abs(Data.Bot_width - bot_w) <= 50)
        condition2 = (Data.Top_width > 0)
        condition3 = (np.abs(Data.Top_width - top_w) <= 50)
        condition4 = (Data.Depth > 0)
        condition5 = (np.abs(Data.Depth - depth) <= 50) 

        index = (np.where(condition0 & condition1 & condition2 & condition3 &
                          condition4 & condition5 ))[0]
        indexes.extend(index.tolist())

        for i in indexes:
            radian2 = np.arctan((Data.iloc[i].Top_width-Data.iloc[i].Bot_width)/(2* Data.iloc[i].Depth))
            case = np.r_[round(np.degrees(radian2),2), (Data.iloc[i].Frequency / 40. ), ((Data.iloc[i]).round(3))]
            result.append(case)  
        DATA = np.array(result)    
        df_results = pd.DataFrame(DATA,
                                  columns = 
    ['angle', 'Speed', 'Frequency','Amplitude', 'Passes', 'Focal_Position', 'Depth',
       'Top_width', 'Bot_width'])
        return df_results

In [34]:
GAN_result = pd.read_csv('RawData/two_widths/GANtune_NNPRed.csv')

result = ParamTable_TargetTrap(GAN_result, 444.5, 500, 100)


result = result[(result > 0).all(1)]

result.to_csv(r'RawData/two_widths/TestData/Trap_GAN.csv', header=True, index = False)

result

,angle,Speed,Frequency,Amplitude,Passes,Focal_Position,Depth,Top_width,Bot_width
0,24.76,17.329925,693.197,0.531,26.0,92.148,448.365,534.795,121.205
1,22.80,21.939450,877.578,0.497,20.0,93.198,443.159,498.748,126.246
2,25.51,39.673750,1586.950,0.459,29.0,93.200,398.319,463.812,83.748
3,21.01,15.106200,604.248,0.530,18.0,93.169,488.654,517.009,141.572
4,23.58,7.914050,316.562,0.441,14.0,93.028,400.492,491.638,142.101
...,...,...,...,...,...,...,...,...,...
60,22.76,22.041350,881.654,0.492,20.0,93.199,443.638,493.902,121.580
61,23.35,26.171650,1046.866,0.462,22.0,93.200,422.875,469.035,103.854
62,22.76,13.416250,536.650,0.530,21.0,92.270,487.569,532.173,123.074
63,22.14,21.464450,858.578,0.455,18.0,93.198,439.031,454.524,97.191


In [27]:
def ParamTable_TargetTriang(Data, depth, top_w):
        result = []
        indexes = []

        condition0 = (Data.Bot_width > 0)
        condition1 = (np.abs(Data.Bot_width) <= 10)
        condition2 = (Data.Top_width > 0) 
        condition3 = (np.abs(Data.Top_width-top_w) <= 50)
        condition4 = (Data.Depth > 0) 
        condition5 = (np.abs(Data.Depth-depth) <= 50)

        index = (np.where(condition0 & condition1 & condition2 & condition3 &
                          condition4 & condition5))[0]
        indexes.extend(index.tolist())

        for i in indexes:
            radian2 = np.arctan((Data.iloc[i].Top_width-Data.iloc[i].Bot_width)/(2* Data.iloc[i].Depth))
            case = np.r_[round(np.degrees(radian2),2),Data.iloc[i].Frequency / 40., ((Data.iloc[i]).round(3))]
            result.append(case)  
        DATA = np.array(result)    
        df_results = pd.DataFrame(DATA, columns = 
    ['angle', 'Speed','Frequency', 'Amplitude', 'Passes', 'Focal_Position', 'Depth',
       'Top_width', 'Bot_width'])
        return df_results

In [36]:
GAN_result = pd.read_csv('RawData/two_widths/GANtune_NNPRed.csv')

result = ParamTable_TargetTriang(GAN_result, 635, 500)


result = result[(result > 0).all(1)]

result.to_csv(r'RawData/two_widths/TestData/Triang_GAN.csv', header=True, index = False)

result

,angle,Speed,Frequency,Amplitude,Passes,Focal_Position,Depth,Top_width,Bot_width
0,22.68,21.183025,847.321,0.531,38.0,93.199,631.747,535.666,7.681
1,20.45,21.077700,843.108,0.461,79.0,93.198,633.785,472.865,0.181
2,22.59,24.843525,993.741,0.517,88.0,93.200,636.473,536.725,7.087
3,20.95,21.229800,849.192,0.497,88.0,93.200,662.365,509.152,1.921
4,22.13,21.129725,845.189,0.494,35.0,93.198,609.168,496.680,1.284
5,20.69,20.609175,824.367,0.495,88.0,93.198,669.813,506.242,0.210
6,22.65,25.186900,1007.476,0.504,77.0,93.200,624.526,523.734,2.636
7,22.71,21.963925,878.557,0.519,38.0,93.200,620.218,525.140,5.884
8,22.69,21.152975,846.119,0.537,40.0,93.200,643.290,542.658,4.661
9,20.63,21.211450,848.458,0.485,99.0,93.200,656.121,495.521,1.612
